In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load pre-trained YOLO model and coco class names
yolo_net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in yolo_net.getUnconnectedOutLayers()]

# Load car color classification model
color_model = load_model('car_color_model.h5')

# Load gender classification model
gender_model = load_model('gender_classification_model.h5')

def detect_objects(image):
    height, width, channels = image.shape
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    outs = yolo_net.forward(output_layers)
    
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    return boxes, confidences, class_ids, indexes

def classify_car_color(image):
    resized_image = cv2.resize(image, (64, 64))
    resized_image = resized_image.astype('float32') / 255.0
    resized_image = np.expand_dims(resized_image, axis=0)
    prediction = color_model.predict(resized_image)
    return np.argmax(prediction)

def classify_gender(person_image):
    resized_image = cv2.resize(person_image, (64, 64))
    resized_image = resized_image.astype('float32') / 255.0
    resized_image = np.expand_dims(resized_image, axis=0)
    prediction = gender_model.predict(resized_image)
    return 'male' if prediction < 0.5 else 'female'

def process_image(image_path):
    image = cv2.imread(image_path)
    boxes, confidences, class_ids, indexes = detect_objects(image)
    
    car_count = 0
    other_vehicles_count = 0
    male_count = 0
    female_count = 0
    
    for i in indexes:
        i = i[0]
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        
        if label == "car":
            car_count += 1
            car_image = image[y:y+h, x:x+w]
            color_id = classify_car_color(car_image)
            # Swap red and blue
            if color_id == 0:  # assuming 0 is red and 1 is blue
                color_id = 1
            elif color_id == 1:
                color_id = 0
            print(f"Car color: {'red' if color_id == 0 else 'blue'}")
        elif label in ["truck", "bus", "motorbike", "bicycle"]:
            other_vehicles_count += 1
        elif label == "person":
            gender = classify_gender(image[y:y+h, x:x+w])
            if gender == "male":
                male_count += 1
            else:
                female_count += 1
    
    print(f"Total cars: {car_count}")
    print(f"Other vehicles: {other_vehicles_count}")
    print(f"Males: {male_count}, Females: {female_count}")

# Test the process_image function
process_image('test_image.jpg')
